In [1]:
from keras.preprocessing.text import Tokenizer
from numpy import array
from keras.utils import to_categorical
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
data= """Jack and Jill went up the hill \n
To fetch a pill of water\n
Jack fell down and broke his crown\n
And Jill came trembling after\n"""

In [3]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])
encoded=tokenizer.texts_to_sequences([data])[0]
vocab_size=len(tokenizer.word_index)+1

In [15]:
sequences=[]
for i in range(2,len(encoded)):
    sequence=encoded[i-2:i+1]
    sequences.append(sequence)
print(len(sequences))
max_length=max([len(seq) for seq in sequences])
sequences=pad_sequences(sequences, maxlen=max_length, padding='pre')
sequences=array(sequences)
X, y= sequences[:,:-1],sequences[:,-1]
y=to_categorical(y,num_classes=vocab_size)

23


In [16]:
def define_model(vocab_size,max_length):
    model= Sequential()
    model.add(Embedding(vocab_size, 10, input_length=max_length-1))
    model.add(LSTM(50))
    model.add(Dense(vocab_size,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [17]:
print(X.shape, y.shape)

(23, 2) (23, 22)


In [18]:
model=define_model(vocab_size, max_length)
model.fit(X,y, epochs=500, verbose=2)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 2, 10)             220       
                                                                 
 lstm_2 (LSTM)               (None, 50)                12200     
                                                                 
 dense_2 (Dense)             (None, 22)                1122      
                                                                 
Total params: 13542 (52.90 KB)
Trainable params: 13542 (52.90 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500


1/1 - 2s - loss: 3.0911 - accuracy: 0.0870 - 2s/epoch - 2s/step
Epoch 2/500
1/1 - 0s - loss: 3.0902 - accuracy: 0.1304 - 16ms/epoch - 16ms/step
Epoch 3/500
1/1 - 0s - loss: 3.0893 - accuracy: 0.0870 - 14ms/epoch - 14ms/step
Epoch 4/500
1/1 - 0s - loss: 3.0884 - accura

In [26]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text, result= seed_text, seed_text
    for _ in range(n_words):
        encoded=tokenizer.texts_to_sequences([in_text])[0]
        # encoded=array(encoded)
        encoded=pad_sequences([encoded],maxlen=max_length, padding='pre')
        yhat=model.predict(encoded, verbose=0)
        predicted_class= np.argmax(yhat,axis=1)
        out_word=''
        for word, index in tokenizer.word_index.items():
            if index == predicted_class:
                out_word=word
                break
        in_text,result = out_word, result+ ' '+ out_word
        return result

In [27]:
print(generate_seq(model, tokenizer, max_length-1,  'Jack and',5 ))

Jack and jill
